# 1 - Tiling

In [ ]:
# tiling parameters

tile_size_m=10          # tile size in m
buffer_size_m=0         # tile buffer size_m (used to produce overlapping tiles)

In [ ]:
# path to own google drive

my_drive_path="/content/drive/MyDrive"
path_data=my_drive_path+"/NOVA_course_deep_learning/data"

# path to the drone mosaic

input_ortho_path=path_data+"/orthomosaics/ortho_hobol_042222_mavic_sun.tif"


### 1.1 Installing and importing required python libraries

In [ ]:
!pip install geopandas
!pip install rasterio
!pip install folium matplotlib mapclassify

In [ ]:
# general python packages

import os, glob, shutil
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# geospatial packages

from osgeo import gdal, ogr, osr
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio
import folium

# define path to where osgeo utils (gdal_retile.py) are stored

path_osgeo_utils= "/usr/local/lib/python3.10/dist-packages/osgeo_utils"

### 1.2 Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1.3 Loading data

#### Defining paths to available orthomosaics

In [ ]:
path_data=my_drive_path+"/NOVA_course_deep_learning/data/"
orthos = os.listdir(path_data+"/orthomosaics")

#### Loading orthomosaic footprints

In [ ]:
# directory to where the drone acquisition geojson is stored

dir_drone_acquisitions=path_data+"/map_data/drone_acquisitions.geojson"
dir_drone_acquisitions

In [ ]:
footprints= gpd.read_file(dir_drone_acquisitions)

In [ ]:
# data location on a map
footprints.explore()

### 1.4 Tiling

In [ ]:
# get orthomosaic name

ortho_name=os.path.splitext(os.path.basename(input_ortho_path)) [0]
ortho_name

Defining and creating an output directory to store the tiles

In [ ]:
output_tiles_dir=path_data+"tiles/"+str(tile_size_m)+"m_"+ortho_name
if not os.path.exists(output_tiles_dir):
    os.makedirs(output_tiles_dir)

output_tiles_dir

In [ ]:
# pixel resolution (in meters) and tile size in pixels

src_ds = gdal.Open(input_ortho_path)                # reads in the orthomosaic
_, xres, _, _, _, yres  = src_ds.GetGeoTransform()  # get pixel size in meters
print("Ortho resolution: "+str(round(xres,4))+" m")

# EPSG code

proj = osr.SpatialReference(wkt=src_ds.GetProjection())
EPSG_code= proj.GetAttrValue('AUTHORITY',1)
print("EPSG code: "+str(EPSG_code))

# number of bands

n_bands=src_ds.RasterCount
print("Number of bands: "+str(n_bands))

In [ ]:
# Compute tile and buffer size in pixels

tile_size_px= round(tile_size_m/abs(xres))        # calculate the tile size in pixels
buffer_size_px= round(buffer_size_m/abs(xres))    # calculate the buffer size in pixels
print("The tile size in pixels is: "+str(tile_size_px))
print("The size of the overalp between tiles in pixels is: "+str(buffer_size_px))

In [ ]:
# define name for output tile index shapefile

tileIndex_name=ortho_name+"_tile_index"

# Run gdal_retile.py using CLI

command_retile = "python "+path_osgeo_utils+"/gdal_retile.py -targetDir " + output_tiles_dir + " " + input_ortho_path+ " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of GTiff -tileIndex "+ tileIndex_name + " -tileIndexField ID"
print(os.popen(command_retile).read())

### 1.5 selecting only tiles that are fully within the footprint polygon

In [ ]:
footprint_ortho= footprints[footprints['filename']==ortho_name]
footprint_ortho_UU= footprint_ortho.geometry.unary_union

In [ ]:
# Load tiles shapefile

tiles = gpd.read_file(output_tiles_dir+ "/"+ortho_name+"_tile_index.shp")
tiles= tiles.to_crs(EPSG_code)

# Select all tiles that are within the boundary polygon

tiles_in = tiles[tiles.geometry.within(footprint_ortho_UU)]

# Select all tiles that are not within the boundary polygon

tiles_out= tiles.loc[~tiles['ID'].isin(tiles_in['ID']) ]
print(str(len(tiles_out))+" tiles to be deleted")
print(str(len(tiles_in))+" tiles remaining")

In [ ]:
# delete tiles that are not within the footprint

gtiffs_delete=[output_tiles_dir+ "/"+sub  for sub in tiles_out['ID']]
for f in gtiffs_delete:
   if os.path.exists(f):
     os.remove(f)

Plot footprint with all of the tiles and the ones that are fully within

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
footprint_ortho.plot(ax=ax, edgecolor="black",facecolor="none")
tiles.plot(ax=ax, edgecolor="red",facecolor="none")
tiles_in.plot(ax=ax, color="green",facecolor="none")

In [ ]:
print("Check in "+ output_tiles_dir+" for exported tiles")